In [11]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [12]:
PROJECT_PATH = r"C:\Users\lenovo i3\OneDrive\Documents\Project Data\Compus x project\filght-price-prediction"
DATA_DIR = "dataset"

In [13]:
def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_PATH, DATA_DIR, file_name)
    return pd.read_csv(file_path)

In [14]:
flights = get_data("flight_price")
flights.sample(10)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
8954,SpiceJet,15/06/2019,Mumbai,Hyderabad,BOM → HYD,05:45,07:15,1h 30m,non-stop,No info,2753
6694,Jet Airways,24/03/2019,Kolkata,Banglore,CCU → BOM → BLR,08:25,22:35,14h 10m,1 stop,In-flight meal not included,10031
4536,SpiceJet,18/06/2019,Kolkata,Banglore,CCU → BLR,22:20,00:40 19 Jun,2h 20m,non-stop,No check-in baggage included,3841
612,Jet Airways,15/06/2019,Delhi,Cochin,DEL → BOM → COK,22:50,19:00 16 Jun,20h 10m,1 stop,In-flight meal not included,10262
376,Multiple carriers,27/03/2019,Delhi,Cochin,DEL → BOM → COK,10:00,18:50,8h 50m,1 stop,No info,11667
5298,SpiceJet,9/06/2019,Kolkata,Banglore,CCU → BLR,09:00,11:25,2h 25m,non-stop,No info,4174
5896,Jet Airways,18/05/2019,Kolkata,Banglore,CCU → BOM → BLR,14:05,10:05 19 May,20h,1 stop,In-flight meal not included,9663
2914,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,10:35,19:15,8h 40m,1 stop,No info,7575
10146,Air India,21/05/2019,Delhi,Cochin,DEL → HYD → BOM → COK,17:15,19:15 22 May,26h,2 stops,No info,11972
890,Multiple carriers,6/03/2019,Delhi,Cochin,DEL → BOM → COK,07:30,21:00,13h 30m,1 stop,No info,15147


In [15]:
flight.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- dataset contain 10683 `Rows` and 11 `Columns`
- `Route` and `Total_stops` columns have 1 missing value
- The datatype of some columns is not apropriate

## 3. Preliminary analysis

### 3.1 check for data types

In [16]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [17]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

In [19]:
value = flights.Date_of_Journey.iloc[5]
value

'24/06/2019'

In [20]:
value = flights.Dep_Time.iloc[6]
value

'18:55'

### 3.2 check for duplicates values

In [21]:
flights.duplicated().sum()

220

In [23]:
(
    flights
    .loc[flights.duplicated(keep=False)]
    .sort_values(["Airline", "Date_of_Journey", "Source", "Destination" ])    
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


### 3.3 check observations

- The type of `Date_of_Journey`, `Dep_Time` and `Arrival_Time` should be changed to datetime
- The type of `Duration` and `Total_Stops` is mixed. It should be numeric type.
- There're 220 duplicates. These should be removed.

## 4. Detailed Analysis

### Airline

In [24]:
flights.Airline

0             IndiGo
1          Air India
2        Jet Airways
3             IndiGo
4             IndiGo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

In [25]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [ ]:
J